# The Arena
### Pitting contenders against each other

In [1]:
from tqdm import tqdm
from domoku.policies.softadvice import MaxInfluencePolicyParams, MaxInfluencePolicy
from domoku.policies.maximal_criticality import MaxCriticalityPolicy
from domoku.policies import softadvice
from alphazero.gomoku_game import GomokuGame as Game
from alphazero.mcts import MCTS
from alphazero.gomoku_board import *

In [2]:
BOARD_SIZE=15
detector = MaxCriticalityPolicy(BOARD_SIZE)
the_game = Game(15, detector=detector, initial='H8')

2022-06-18 17:49:46.946341: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/wgiersche/.local/share/virtualenvs/DeepGomoku-cXtJ_EtM/lib/python3.10/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [3]:
def given_heuristic_brain():
    hard_policy = MaxCriticalityPolicy(board_size=BOARD_SIZE, overconfidence=2.0)
    mi_params = MaxInfluencePolicyParams(
        board_size=BOARD_SIZE,
        sigma=.6,
        iota=6,
        radial_constr=[.0625, .125, .25, .5],
        radial_obstr=[-.0625, -.125, -.25, -.5]
    )
    policy = MaxInfluencePolicy(mi_params, criticality_model=hard_policy, pov=1)
    return softadvice.NeuralNetAdapter(policy)


def longest_trajectory(ts):
    depths = [v.level for v in ts.tree_nodes.values()]
    mx = max(depths)
    return [v for v in ts.tree_nodes.values() if v.level == mx][0]


def summary(ts: MCTS, board: Board):

    return longest_trajectory(ts)

In [4]:
brain_d = given_heuristic_brain()
mcts_d = MCTS(the_game, brain_d, cpuct=1.0, num_simulations=20, model_threshold=.6)

brain_c = given_heuristic_brain()
mcts_c = MCTS(the_game, brain_c, cpuct=1.0, num_simulations=50, model_threshold=.9)

board = the_game.get_initial_board()
Move=board.Stone

In [8]:
probs = mcts_c.get_action_prob(board, temperature=0)
move =Move(probs)
print(move)
board.act(move)
board.plot()
longest = summary(mcts_c, board)
print(longest.key)
print(longest)

G11
                                                     
15    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
14    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
13    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
12    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
11    .  .  .  .  .  . [X] .  .  .  .  .  .  .  .    
10    .  .  .  .  .  .  .  X  .  .  .  .  .  .  .    
 9    .  .  .  .  .  .  .  .  O  .  .  .  .  .  .    
 8    .  .  .  .  .  .  .  X  .  .  .  .  .  .  .    
 7    .  .  .  .  .  .  .  O  .  .  .  .  .  .  .    
 6    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 5    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 4    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 3    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 2    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
 1    .  .  .  .  .  .  .  .  .  .  .  .  .  .  .    
                                                     
      A  B  C  D  E  F  G  H  I  J  K  L  M  N  O
-1742325376912623003
[H8, H7

In [9]:
current_node = mcts_c.tree_nodes[board.get_string_representation()]
print(current_node.key)
current_node

3566410095357358075


[H8, H7, H10, I9, G11]: -0.000505605770740658
          [H8, H7, H10, I9, G11, F10]: -0.00035544356796890497
            [H8, H7, H10, I9, G11, F10, H11]: -0.10000350326299667
              [H8, H7, H10, I9, G11, F10, H11, I11]: -0.19749604165554047
                
              [H8, H7, H10, I9, G11, F10, H11, H12]: -0.19752980768680573
                
          [H8, H7, H10, I9, G11, G10]: -0.00031858269358053803
            [H8, H7, H10, I9, G11, G10, I8]: -0.09982689470052719
              [H8, H7, H10, I9, G11, G10, I8, J8]: -0.0002790181606542319
                
              [H8, H7, H10, I9, G11, G10, I8, K8]: -0.0002640417660586536
                
              [H8, H7, H10, I9, G11, G10, I8, G8]: -0.09994640201330185
                
            [H8, H7, H10, I9, G11, G10, I6]: -0.00024659588234499097
              [H8, H7, H10, I9, G11, G10, I6, J6]: -0.0002854165795724839
                [H8, H7, H10, I9, G11, G10, I6, J6, J7]: -0.0998118445277214
                  [H8,

In [2]:
class Player:
    def __init__(self, mcts: MCTS, temperature):
        self.mcts = mcts
        self.temperature = temperature

    def play(self, board: Board):
       pass

In [3]:
class Arena:
    def __init__(self, game: Game):
        self.game = game
        pass

    def play_game(self, black: Player, white: Player, verbose: bool):
        pass

    def play_games(self, player1: Player, player2: Player, num_games, verbose=False):
        """
        Plays num games in which player1 starts num/2 games and player2 starts
        num/2 games.

        Returns:
            one_won: games won by player1
            two_won: games won by player2
            draws:  games won by nobody
        """

        num = int(num_games / 2)
        one_won = 0
        two_won = 0
        n_draws = 0
        for _ in tqdm(range(num), desc="Arena.play_games (1)"):
            game_result = self.play_game(black=player1, white=player2, verbose=verbose)
            if game_result == 1:
                one_won += 1
            elif game_result == -1:
                two_won += 1
            else:
                n_draws += 1

        for _ in tqdm(range(num), desc="Arena.play_games (2)"):
            game_result = self.play_game(black=player2, white=player1, verbose=verbose)
            if game_result == -1:
                one_won += 1
            elif game_result == 1:
                two_won += 1
            else:
                n_draws += 1

        return one_won, two_won, n_draws



In [ ]:
arena = Arena(the_game)

defender = Player(mcts_d, temperature=0)

challenger = Player(mcts_c, temperature=0)

defender_wins, challenger_wins, draws = arena.play_games(num_games=10, player1=defender, player2=challenger)

